In [44]:
import re
from os import listdir
from os.path import isfile, join
import random
import json

In [59]:
def split_script_into_json(src, dest, unique_id, block_count, test_count):
    with open(src, "r") as f:
        lines = f.readlines()
        line_count = len(lines)
        lines_per_block = line_count // block_count
        blocks = random.sample(range(0, block_count), test_count)
        data = {train":""}
        test_itr = 1
        for i in range(block_count):
            start_index = i*lines_per_block
            end_index = start_index + lines_per_block
            split_index = (start_index + end_index) // 2
            
            if i in blocks:
                data["test"+str(test_itr)+"_input"] = ''.join(lines[start_index:split_index])
                data["test"+str(test_itr)+"_true"]  = ''.join(lines[split_index+1:end_index])
                data["test"+str(test_itr)+"_id"]    = unique_id + test_itr - 1
                test_itr += 1
            else:
                data["train"] += ''.join(lines[start_index:split_index])
        print(data.keys())
        dest = dest[:-4] + ".json"
        with open(dest, 'w+') as outfile:
            json.dump(data, outfile)
            

def generate_test_jsons_entire_dir(srcpath, destpath):
    unique_id = 0
    blocks_per_file = 20
    tests_per_file = 3
    for f in listdir(srcpath):
        src = join(srcpath, f)
        dest = join(destpath, f)
        if isfile(src):
            split_script_into_json(src, dest, unique_id, blocks_per_file, tests_per_file)
        unique_id += tests+per_file
    
def parse_script(src, dest):
    with open(src, "r") as f:
        parsed = ""
        added_new = True # Used for ignoring consequtive empty lines
        for line in f:
            temp = line.strip(" ")
            if temp != "\n":
                temp = temp.strip("\n")
                if (re.search('[a-zA-Z]', temp)): # Ignore page number lines
                    parsed += line.strip() + " "
                    added_new = False
            else:
                if not added_new:
                    parsed += "\n"
                    added_new = True
                continue
    with open(dest, "w") as f:
        f.write(parsed)

def process_entire_dir(srcpath, destpath):
    for f in listdir(srcpath):
        src = join(srcpath, f)
        dest = join(destpath, f)
        if isfile(src):
            parse_script(src, dest)
    print("Processed: {} scripts".format(len(listdir(srcpath))))

SyntaxError: invalid syntax (<ipython-input-59-78775b0ff401>, line 7)

In [ ]:
process_entire_dir("./raw_data/", "./parsed_data/")

In [60]:
generate_test_jsons_entire_dir("./parsed_data/", "./test_cases/")

dict_keys(['id', 'train', 'test1_input', 'test1_true', 'test1_id', 'test2_input', 'test2_true', 'test2_id', 'test3_input', 'test3_true', 'test3_id'])


TypeError: Object of type builtin_function_or_method is not JSON serializable